In [1]:
from datetime import datetime, timedelta
import requests
import json
from time import sleep

def generate_date_range(year_from, month_from, day_from, year_to, month_to, day_to, interval):
    
    date_from = datetime(year_from, month_from, day_from).date()
    date_to = datetime(year_to, month_to, day_to).date()

    month_years = []
    while date_from < date_to:
        month_years.append(date_from.strftime('%Y%m')+'01T0000')
        date_from += timedelta(days=interval)
    
    return month_years

def convert_date_string(date_string):
    date_format = '%Y%m%dT%H%M%S'
    return datetime.strptime(date_string, date_format).date()


In [2]:
def get_news_sentiment(api_key, function, tickers, time_from, time_to, limit):
    url = 'https://www.alphavantage.co/query'
    
    parameters = {
        'function': function,
        'tickers': tickers,
        'time_from': time_from,
        'time_to': time_to,
        'limit': limit,
        'apikey': api_key
    }
    
    try:
        response = requests.get(url, params=parameters)
        response.raise_for_status()
        news_list = response.json()['feed']
    except (requests.exceptions.HTTPError, json.JSONDecodeError) as e:
        print(f'An error occurred: {e}')
        news_list = []
    
    if news_list:
        title = [news['title'] for news in news_list]
        summary = [news['summary'] for news in news_list]
        image = [news['banner_image'] for news in news_list]
        source = [news['source'] for news in news_list]
        topics = [news['topics'] for news in news_list]
        overall_sentiment_score = [news['overall_sentiment_score'] for news in news_list]
        overall_sentiment_label = [news['overall_sentiment_label'] for news in news_list]
        ticker_sentiment = [news['ticker_sentiment'] for news in news_list]
        dates = [convert_date_string(news['time_published']) for news in news_list]
    else:
        title, summary, image, source, topics, overall_sentiment_score, overall_sentiment_label, ticker_sentiment, dates = [], [], [], [], [], [], [], [], []
    
    
    return title, summary, image, source, topics, overall_sentiment_score, overall_sentiment_label, ticker_sentiment, dates


In [3]:
import datetime

start_date = datetime.date(2013, 12, 1)
end_date = datetime.date(2023, 3, 1)

delta = datetime.timedelta(days=1)
dates = []

while start_date <= end_date:
    year = start_date.year
    month = start_date.month
    string = f"{year}{month:02d}01T0000"
    dates.append(string)
    start_date += datetime.timedelta(days=32)
    start_date = start_date.replace(day=1)

dates.reverse()

In [76]:
path_key = 'key.json'
apikey = json.load(open(path_key))
datasets = []

for date in range(len(dates)):
    try:
        if date+1 < len(dates):
            time_to = dates[date]
            time_from = dates[date+1]

        news = get_news_sentiment(
            api_key = apikey['key'], 
            function = 'NEWS_SENTIMENT', 
            tickers = 'COIN,CRYPTO:BTC,FOREX:USD', 
            time_from = time_from, 
            time_to = time_to, 
            limit = 200
            )
        
        sleep(2)
        
        datasets.append(news)

    except:
        continue

In [2]:
dicio = {'title':title,
    'summary':summary,
    'image':image,
    'source':source,
    'topics':topics,
    'overall_sentiment_score':overall_sentiment_score,
    'overall_sentiment_label':overall_sentiment_label,
    'ticker_sentiment':ticker_sentiment,
    'dates':dates
}

In [3]:
import pandas as pd

pd.DataFrame(dicio)

,title,summary,image,source,topics,overall_sentiment_score,overall_sentiment_label,ticker_sentiment,dates
0,Crypto Biz: Coinbase has a lot at stake,Coinbase says its staking product offerings ar...,https://s3.cointelegraph.com/uploads/2023-02/5...,Cointelegraph,"[{'topic': 'Financial Markets', 'relevance_sco...",-0.002413,Neutral,"[{'ticker': 'MA', 'relevance_score': '0.111747...",2023-02-24
1,The SEC's Stablecoin Crackdown Could Reshape T...,• The SEC has ordered Paxos to stop trading it...,https://imageio.forbes.com/specials-images/ima...,Forbes,"[{'topic': 'Economy - Monetary', 'relevance_sc...",-0.036424,Neutral,"[{'ticker': 'COIN', 'relevance_score': '0.0696...",2023-02-21
2,What Is A Stablecoin? | The Beginner's Guide,Stablecoins are cryptocurrencies that claim to...,https://img.decrypt.co/insecure/rs:fill:1024:5...,Decrypt.co,"[{'topic': 'Economy - Monetary', 'relevance_sc...",0.186118,Somewhat-Bullish,"[{'ticker': 'BACHY', 'relevance_score': '0.025...",2023-02-21
3,Can you trust crypto exchanges after the colla...,"On Oct. 25, 2022 - about two weeks before the ...",https://cointelegraph.com/magazine/wp-content/...,Cointelegraph,"[{'topic': 'Finance', 'relevance_score': '1.0'...",-0.065094,Neutral,"[{'ticker': 'COIN', 'relevance_score': '0.0606...",2023-02-15
4,Bitcoin price clings to $22K as investors dige...,"Bitcoin price recaptured the $22,000 level, bu...",https://s3.cointelegraph.com/uploads/2023-02/7...,Cointelegraph,"[{'topic': 'Economy - Monetary', 'relevance_sc...",-0.061523,Neutral,"[{'ticker': 'COIN', 'relevance_score': '0.0528...",2023-02-14
5,AI Hits The Mainstream And January Jobs Report...,Subscribe to the Forbes AI newsletter to stay ...,https://imageio.forbes.com/specials-images/ima...,Forbes,"[{'topic': 'Technology', 'relevance_score': '0...",0.097882,Neutral,"[{'ticker': 'MSFT', 'relevance_score': '0.1008...",2023-02-13
6,The 1 Cryptocurrency I'd Buy Right Now,This is an easy decision to make.,https://g.foolcdn.com/image/?url=https%3A%2F%2...,Motley Fool,"[{'topic': 'Economy - Monetary', 'relevance_sc...",0.363459,Bullish,"[{'ticker': 'HOOD', 'relevance_score': '0.0584...",2023-02-12
7,Bitcoin price correction was overdue - Analyst...,BTC and the crypto market will continue to bat...,https://s3.cointelegraph.com/uploads/2023-02/d...,Cointelegraph,"[{'topic': 'Economy - Monetary', 'relevance_sc...",0.122364,Neutral,"[{'ticker': 'COIN', 'relevance_score': '0.0472...",2023-02-10
8,New Hampshire could become an alternative for ...,A state commission recommended that New Hampsh...,https://s3.cointelegraph.com/uploads/2023-02/d...,Cointelegraph,"[{'topic': 'Finance', 'relevance_score': '1.0'...",0.014781,Neutral,"[{'ticker': 'WFC', 'relevance_score': '0.03961...",2023-02-09
9,Top cryptocurrency news on February 7: BabyDog...,A daily round-up of the most interesting artic...,https://images.moneycontrol.com/static-mcnews/...,Money Control,"[{'topic': 'Financial Markets', 'relevance_sco...",0.096392,Neutral,"[{'ticker': 'MSTR', 'relevance_score': '0.0274...",2023-02-07
